# Library

In [1]:
import tabula, math, PyPDF2, json, requests, time, os, glob
import pandas as pd

from tqdm.notebook import tqdm

In [3]:
pdf_tabel = tabula.read_pdf("Salinan Lampiran 3 Kepmendikbudristek BOS.pdf", pages="all")

java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: Java heap space

In [3]:
# Open the PDF file
with open("Salinan Lampiran 3 Kepmendikbudristek BOS.pdf", 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    # Get the number of pages
    total_pages = len(pdf_reader.pages)
    print(f"Total pages in PDF: {total_pages}")

Total pages in PDF: 5954


In [4]:
pdf_info = tabula.read_pdf("Salinan Lampiran 3 Kepmendikbudristek BOS.pdf", pages='1')
#total_pages = 5000
chunk_size = 50
total_chunks = math.ceil(total_pages / chunk_size)
print(f"Total chunks needed: {total_chunks}")


Total chunks needed: 120


c:\Users\aansubarkah\anaconda3\envs\basangdata12\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [5]:
all_tables = []
for chunk in tqdm(range(total_chunks), desc="Processing PDF chunks"):
    start_page = chunk * chunk_size + 1
    end_page = min((chunk + 1) * chunk_size, total_pages)
    
    try:
        # Read chunk of pages
        chunk_tables = tabula.read_pdf(
            "Salinan Lampiran 3 Kepmendikbudristek BOS.pdf",
            pages=f"{start_page}-{end_page}"
        )
        # Extend the list with new tables
        all_tables.extend(chunk_tables)
    except Exception as e:
        print(f"Error processing pages {start_page}-{end_page}: {str(e)}")
        continue

# Combine all tables into pdf_tabel
pdf_tabel = pd.concat(all_tables, ignore_index=True)

Processing PDF chunks:   0%|          | 0/120 [00:00<?, ?it/s]

c:\Users\aansubarkah\anaconda3\envs\basangdata12\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [ ]:
pdf_tabel

,Nomor,Provinsi/\rKabupaten/\rKota,Bentuk Satuan Pendidikan,Nomor Pokok\rSekolah\rNasional,Nama Satuan Pendidikan,Jumlah\rPeserta Didik,Satuan Biaya\r(Rupiah),Besaran Alokasi Dana Satu\rTahun (Rupiah)
0,(a),(b),(c),(d),(e),(f),(g),(h)
1,1,Provinsi Aceh,NaN,NaN,882.520,NaN,1.041.073.930.000,NaN
2,NaN,Kabupaten Aceh Barat,NaN,NaN,30.042,NaN,38.930.680.000,NaN
3,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43,1.010.000,43.430.000
4,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97,1.010.000,97.970.000
...,...,...,...,...,...,...,...,...
226200,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208,1.600.000,332.800.000
226201,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197,1.600.000,315.200.000
226202,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121,3.500.000,423.500.000
226203,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154,3.500.000,539.000.000


In [7]:
# Save to pickle format
pdf_tabel.to_pickle('pdf_tabel.pkl')

# Save to feather format
pdf_tabel.to_feather('pdf_tabel.feather')

In [8]:
pdf_tabel.columns = [
    'nomor',
    'wilayah',
    'bentuk_pendidikan',
    'npsn',
    'nama_sekolah',
    'jumlah_siswa',
    'satuan_biaya',
    'total_alokasi'
]

In [9]:
pdf_tabel

,nomor,wilayah,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi
0,(a),(b),(c),(d),(e),(f),(g),(h)
1,1,Provinsi Aceh,NaN,NaN,882.520,NaN,1.041.073.930.000,NaN
2,NaN,Kabupaten Aceh Barat,NaN,NaN,30.042,NaN,38.930.680.000,NaN
3,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43,1.010.000,43.430.000
4,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97,1.010.000,97.970.000
...,...,...,...,...,...,...,...,...
226200,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208,1.600.000,332.800.000
226201,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197,1.600.000,315.200.000
226202,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121,3.500.000,423.500.000
226203,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154,3.500.000,539.000.000


In [10]:
pdf_tabel = pdf_tabel.drop(0)
pdf_tabel = pdf_tabel.reset_index(drop=True)

In [11]:
pdf_tabel

,nomor,wilayah,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi
0,1,Provinsi Aceh,NaN,NaN,882.520,NaN,1.041.073.930.000,NaN
1,NaN,Kabupaten Aceh Barat,NaN,NaN,30.042,NaN,38.930.680.000,NaN
2,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43,1.010.000,43.430.000
3,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97,1.010.000,97.970.000
4,NaN,NaN,Sekolah Dasar,10107830,SD NEGERI 12 MEULABOH,247,1.010.000,249.470.000
...,...,...,...,...,...,...,...,...
226199,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208,1.600.000,332.800.000
226200,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197,1.600.000,315.200.000
226201,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121,3.500.000,423.500.000
226202,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154,3.500.000,539.000.000


In [15]:
# Remove dots and convert to numeric
pdf_tabel['jumlah_siswa'] = pd.to_numeric(pdf_tabel['jumlah_siswa'].str.replace('.', ''), errors='coerce')
pdf_tabel['satuan_biaya'] = pd.to_numeric(pdf_tabel['satuan_biaya'].str.replace('.', ''), errors='coerce')
pdf_tabel['total_alokasi'] = pd.to_numeric(pdf_tabel['total_alokasi'].str.replace('.', ''), errors='coerce')

In [17]:
# Save to pickle format
pdf_tabel.to_pickle('pdf_tabel_1.pkl')

# Save to feather format
pdf_tabel.to_feather('pdf_tabel_1.feather')

In [19]:
pdf_tabel

,nomor,wilayah,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi,province
0,1,Provinsi Aceh,NaN,NaN,882.520,NaN,1.041074e+12,NaN,Provinsi Aceh
1,NaN,Kabupaten Aceh Barat,NaN,NaN,30.042,NaN,3.893068e+10,NaN,Provinsi Aceh
2,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43.0,1.010000e+06,43430000.0,Provinsi Aceh
3,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97.0,1.010000e+06,97970000.0,Provinsi Aceh
4,NaN,NaN,Sekolah Dasar,10107830,SD NEGERI 12 MEULABOH,247.0,1.010000e+06,249470000.0,Provinsi Aceh
...,...,...,...,...,...,...,...,...,...
226199,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208.0,1.600000e+06,332800000.0,Provinsi Sumatera Utara
226200,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197.0,1.600000e+06,315200000.0,Provinsi Sumatera Utara
226201,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121.0,3.500000e+06,423500000.0,Provinsi Sumatera Utara
226202,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154.0,3.500000e+06,539000000.0,Provinsi Sumatera Utara


In [18]:
# Create a new column 'province'
pdf_tabel['province'] = None

# Initialize current_province
current_province = None

# Iterate through rows
for idx, row in pdf_tabel.iterrows():
    if isinstance(row['wilayah'], str) and 'Provinsi' in row['wilayah']:
        current_province = row['wilayah']
    pdf_tabel.at[idx, 'province'] = current_province

In [20]:
pdf_tabel

,nomor,wilayah,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi,province
0,1,Provinsi Aceh,NaN,NaN,882.520,NaN,1.041074e+12,NaN,Provinsi Aceh
1,NaN,Kabupaten Aceh Barat,NaN,NaN,30.042,NaN,3.893068e+10,NaN,Provinsi Aceh
2,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43.0,1.010000e+06,43430000.0,Provinsi Aceh
3,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97.0,1.010000e+06,97970000.0,Provinsi Aceh
4,NaN,NaN,Sekolah Dasar,10107830,SD NEGERI 12 MEULABOH,247.0,1.010000e+06,249470000.0,Provinsi Aceh
...,...,...,...,...,...,...,...,...,...
226199,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208.0,1.600000e+06,332800000.0,Provinsi Sumatera Utara
226200,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197.0,1.600000e+06,315200000.0,Provinsi Sumatera Utara
226201,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121.0,3.500000e+06,423500000.0,Provinsi Sumatera Utara
226202,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154.0,3.500000e+06,539000000.0,Provinsi Sumatera Utara


In [21]:
# Create a new column 'province'
pdf_tabel['regency'] = None

# Initialize current_province
current_regency = None

# Iterate through rows
for idx, row in pdf_tabel.iterrows():
    if isinstance(row['wilayah'], str) and ('Kabupaten' in row['wilayah'] or 'Kota' in row['wilayah']):
        current_regency = row['wilayah']
    pdf_tabel.at[idx, 'regency'] = current_regency

In [22]:
pdf_tabel

,nomor,wilayah,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi,province,regency
0,1,Provinsi Aceh,NaN,NaN,882.520,NaN,1.041074e+12,NaN,Provinsi Aceh,None
1,NaN,Kabupaten Aceh Barat,NaN,NaN,30.042,NaN,3.893068e+10,NaN,Provinsi Aceh,Kabupaten Aceh Barat
2,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43.0,1.010000e+06,43430000.0,Provinsi Aceh,Kabupaten Aceh Barat
3,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97.0,1.010000e+06,97970000.0,Provinsi Aceh,Kabupaten Aceh Barat
4,NaN,NaN,Sekolah Dasar,10107830,SD NEGERI 12 MEULABOH,247.0,1.010000e+06,249470000.0,Provinsi Aceh,Kabupaten Aceh Barat
...,...,...,...,...,...,...,...,...,...,...
226199,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208.0,1.600000e+06,332800000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
226200,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197.0,1.600000e+06,315200000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
226201,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121.0,3.500000e+06,423500000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
226202,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154.0,3.500000e+06,539000000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi


In [27]:
# Remove rows where npsn column contains "(d)"
pdf_tabel = pdf_tabel[pdf_tabel['npsn'] != "(d)"]
# Reset index after removing rows
pdf_tabel = pdf_tabel.reset_index(drop=True)

In [37]:
# Remove rows where bentuk_pendidikan is NaN
pdf_tabel = pdf_tabel[pd.notna(pdf_tabel['bentuk_pendidikan'])]
# Reset index after removing rows 
pdf_tabel = pdf_tabel.reset_index(drop=True)

In [38]:
pdf_tabel

,nomor,wilayah,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi,province,regency
0,NaN,NaN,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43.0,1010000.0,43430000.0,Provinsi Aceh,Kabupaten Aceh Barat
1,NaN,NaN,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97.0,1010000.0,97970000.0,Provinsi Aceh,Kabupaten Aceh Barat
2,NaN,NaN,Sekolah Dasar,10107830,SD NEGERI 12 MEULABOH,247.0,1010000.0,249470000.0,Provinsi Aceh,Kabupaten Aceh Barat
3,NaN,NaN,Sekolah Dasar,10107831,SD NEGERI 13 MEULABOH,126.0,1010000.0,127260000.0,Provinsi Aceh,Kabupaten Aceh Barat
4,NaN,NaN,Sekolah Dasar,10107832,SD NEGERI 14 MEULABOH,440.0,1010000.0,444400000.0,Provinsi Aceh,Kabupaten Aceh Barat
...,...,...,...,...,...,...,...,...,...,...
219694,NaN,NaN,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208.0,1600000.0,332800000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
219695,NaN,NaN,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197.0,1600000.0,315200000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
219696,NaN,NaN,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121.0,3500000.0,423500000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
219697,NaN,NaN,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154.0,3500000.0,539000000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi


In [39]:
# Save to pickle format
pdf_tabel[['wilayah', 'bentuk_pendidikan', 'npsn', 'nama_sekolah', 'jumlah_siswa', 'satuan_biaya', 'total_alokasi', 'province', 'regency']].to_pickle('pdf_tabel_clean.pkl')

# Save to feather format
pdf_tabel[['wilayah', 'bentuk_pendidikan', 'npsn', 'nama_sekolah', 'jumlah_siswa', 'satuan_biaya', 'total_alokasi', 'province', 'regency']].to_feather('pdf_tabel_clean.feather')

In [42]:
df_bos_reguler_2024 = pdf_tabel[['bentuk_pendidikan', 'npsn', 'nama_sekolah', 'jumlah_siswa', 'satuan_biaya', 'total_alokasi', 'province', 'regency']].copy()

In [43]:
df_bos_reguler_2024

,bentuk_pendidikan,npsn,nama_sekolah,jumlah_siswa,satuan_biaya,total_alokasi,province,regency
0,Sekolah Dasar,10107828,SD MUHAMMADIYAH MEULABOH,43.0,1010000.0,43430000.0,Provinsi Aceh,Kabupaten Aceh Barat
1,Sekolah Dasar,10107829,SD NEGERI 11 MEULABOH,97.0,1010000.0,97970000.0,Provinsi Aceh,Kabupaten Aceh Barat
2,Sekolah Dasar,10107830,SD NEGERI 12 MEULABOH,247.0,1010000.0,249470000.0,Provinsi Aceh,Kabupaten Aceh Barat
3,Sekolah Dasar,10107831,SD NEGERI 13 MEULABOH,126.0,1010000.0,127260000.0,Provinsi Aceh,Kabupaten Aceh Barat
4,Sekolah Dasar,10107832,SD NEGERI 14 MEULABOH,440.0,1010000.0,444400000.0,Provinsi Aceh,Kabupaten Aceh Barat
...,...,...,...,...,...,...,...,...
219694,Sekolah Menengah Kejuruan,69840940,SMK SWASTA KESEHATAN GANDA HUSADA,208.0,1600000.0,332800000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
219695,Sekolah Menengah Kejuruan,69916026,SMK SWASTA R.A. KARTINI,197.0,1600000.0,315200000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
219696,Sekolah Luar Biasa,10211532,SLB Negeri Tebing Tinggi,121.0,3500000.0,423500000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi
219697,Sekolah Luar Biasa,69886466,SLB SWASTA YAPSI TEBING TINGGI,154.0,3500000.0,539000000.0,Provinsi Sumatera Utara,Kota Tebing Tinggi


In [44]:
#df_bos_reguler_2024.to_excel('alokasi_bos_reguler_2024.xlsx', index=False)

# Scrape Dapo

In [13]:
S = requests.Session()

In [14]:
initial_url = 'https://dapo.dikdasmen.go.id/pd'

In [15]:
r = S.get(initial_url)

In [18]:
r.status_code

200

In [19]:
r.text

'<!DOCTYPE html>\n<html lang="id">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta http-equiv="Content-Security-Policy" content="upgrade-insecure-requests">\n\n    <title>Data Peserta Didik - Pauddikdasmen</title>\n\n    \n    <link rel="shortcut icon" type="image/x-icon" href="/assets/img/logo-dikdasmen.png">\n\n    <!-- Fonts -->\n    <link rel="stylesheet" type="text/css" href="http://fonts.googleapis.com/css?family=Roboto:300,400,500,700">\n    <link rel="stylesheet" type="text/css" href="https://fonts.googleapis.com/icon?family=Material+Icons">\n    <link rel=\'stylesheet\' type="text/css" href=\'https://fonts.googleapis.com/css?family=Titillium+Web:400,600,700,300\'>\n\n    <!-- Main Style -->\n    <link rel="stylesheet" type="text/css" href="/assets/libs/bootstrap/css/bootstrap.min.css">\n    <link rel="stylesheet" type="text/css" href="/assets/libs

In [26]:
api_url = 'https://dapo.dikdasmen.go.id/rekap/dataPD?id_level_wilayah='
semester_id = '20231'

In [27]:
api_url_lvl_1 = api_url + '0&kode_wilayah=000000&semester_id=' + semester_id

In [32]:
r1 = S.get(api_url_lvl_1)

In [33]:
df_lvl_1 = pd.DataFrame(r1.json())

In [34]:
df_lvl_1.head()

,nama,kode_wilayah,id_level_wilayah,mst_kode_wilayah,induk_provinsi,kode_wilayah_induk_provinsi,induk_kabupaten,kode_wilayah_induk_kabupaten,tk_pd_laki,tk_pd_perempuan,...,slb_pd,sd_pd_laki,sd_pd_perempuan,sd_pd,smp_pd_laki,smp_pd_perempuan,smp_pd,total_pd_laki,total_pd_perempuan,total_pd
0,Prov. Jawa Barat,020000,1,0,None,None,None,None,198919,187421,...,28472,2317852,2164415,4482267,929485,883626,1813111,5018544,4678803,9697347
1,Prov. Jawa Timur,050000,1,0,None,None,None,None,396694,373764,...,22075,1311980,1217883,2529863,643101,586164,1229265,3419713,3136541,6556254
2,Prov. Jawa Tengah,030000,1,0,None,None,None,None,295578,275191,...,20968,1344604,1248232,2592836,619877,587486,1207363,3200581,2981999,6182580
3,Prov. Sumatera Utara,070000,1,0,None,None,None,None,65149,61177,...,5749,791690,731598,1523288,318217,303554,621771,1639085,1531666,3170751
4,Prov. Banten,280000,1,0,None,None,None,None,54230,51886,...,6678,623393,579371,1202764,227323,214006,441329,1267714,1176642,2444356


In [35]:
df_lvl_1.to_feather('dapo_20231_lvl_1.feather')

## Scrape Lvl 2

In [ ]:
all_kode_wilayah = []
ls_df_lvl_2 = []

for _, row in tqdm(df_lvl_1.iterrows(), desc="Processing provinces", total=len(df_lvl_1)):
    kode_wilayah = row['kode_wilayah']
    api_url_lvl_2 = api_url + '1&kode_wilayah=' + kode_wilayah.strip() + '&semester_id=' + semester_id
    r2 = S.get(api_url_lvl_2)
    #print(r2.text)
    temp_df_lvl_2 = pd.DataFrame(r2.json())
    ls_df_lvl_2.append(temp_df_lvl_2)
    all_kode_wilayah.append(kode_wilayah)
    time.sleep(1)

Processing provinces:   0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
df_lvl_2 = pd.concat(ls_df_lvl_2, ignore_index=True)

In [ ]:
df_lvl_2

,nama,kode_wilayah,id_level_wilayah,mst_kode_wilayah,induk_provinsi,kode_wilayah_induk_provinsi,induk_kabupaten,kode_wilayah_induk_kabupaten,tk_pd_laki,tk_pd_perempuan,...,slb_pd,sd_pd_laki,sd_pd_perempuan,sd_pd,smp_pd_laki,smp_pd_perempuan,smp_pd,total_pd_laki,total_pd_perempuan,total_pd
0,Kab. Bogor,020500,2,020000,Prov. Jawa Barat,020000,None,None,11647,11084,...,698,284737,265407,550144,122134,111322,233456,603763,550064,1153827
1,Kab. Bandung,020800,2,020000,Prov. Jawa Barat,020000,None,None,12879,12083,...,3495,183417,172908,356325,69793,68642,138435,357584,341818,699402
2,Kab. Bekasi,022200,2,020000,Prov. Jawa Barat,020000,None,None,18684,17482,...,715,181354,170589,351943,67609,64767,132376,351316,331287,682603
3,Kab. Garut,021100,2,020000,Prov. Jawa Barat,020000,None,None,12982,12092,...,2341,138479,130625,269104,53355,51324,104679,307072,289840,596912
4,Kab. Cianjur,020700,2,020000,Prov. Jawa Barat,020000,None,None,5554,5099,...,808,133158,125202,258360,49827,47185,97012,286230,267485,553715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Filipina,350600,2,350000,Luar Negeri,350000,None,None,0,0,...,0,13,10,23,14,17,31,43,49,92
522,Cina,351400,2,350000,Luar Negeri,350000,None,None,0,0,...,0,0,0,0,0,0,0,4,49,53
523,Myanmar,350500,2,350000,Luar Negeri,350000,None,None,0,0,...,0,7,7,14,2,5,7,13,12,25
524,Brunei Darussalam,351500,2,350000,Luar Negeri,350000,None,None,0,0,...,0,0,0,0,0,0,0,5,6,11


In [ ]:
df_lvl_2.to_feather('dapo_20231_lvl_2.feather')

## Scrape Lvl 3

In [37]:
ls_df_lvl_3 = []

for _, row in tqdm(df_lvl_2.iterrows(), desc="Processing regencies", total=len(df_lvl_2)):
    kode_wilayah = row['kode_wilayah']
    api_url_lvl_3 = api_url + '2&kode_wilayah=' + kode_wilayah.strip() + '&semester_id=' + semester_id
    r3 = S.get(api_url_lvl_3)
    #print(r2.text)
    temp_df_lvl_3 = pd.DataFrame(r3.json())
    ls_df_lvl_3.append(temp_df_lvl_3)
    #all_kode_wilayah.append(kode_wilayah)
    time.sleep(0.5)

Processing provinces:   0%|          | 0/526 [00:00<?, ?it/s]

In [38]:
df_lvl_3 = pd.concat(ls_df_lvl_3, ignore_index=True)

In [39]:
df_lvl_3

,nama,kode_wilayah,id_level_wilayah,mst_kode_wilayah,induk_provinsi,kode_wilayah_induk_provinsi,induk_kabupaten,kode_wilayah_induk_kabupaten,tk_pd_laki,tk_pd_perempuan,...,slb_pd,sd_pd_laki,sd_pd_perempuan,sd_pd,smp_pd_laki,smp_pd_perempuan,smp_pd,total_pd_laki,total_pd_perempuan,total_pd
0,Kec. Cibinong,020521,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,1453,1395,...,379,19012,17629,36641,8673,8314,16987,44507,41693,86200
1,Kec. Cileungsi,020518,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,1163,1142,...,136,19021,18022,37043,7798,7379,15177,38823,36237,75060
2,Kec. Gunungputri,020519,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,1057,978,...,0,16153,14662,30815,6060,5714,11774,31183,28051,59234
3,Kec. Bojong Gede,020522,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,977,905,...,0,12985,12272,25257,5250,4957,10207,27075,25475,52550
4,Kec. Citeureup,020520,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,421,479,...,0,10293,9698,19991,4558,4146,8704,21680,20046,41726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7264,Hong Kong,351401,3,351400,Luar Negeri,350000,Cina,351400,0,0,...,0,0,0,0,0,0,0,4,49,53
7265,Beijing,351402,3,351400,Luar Negeri,350000,Cina,351400,0,0,...,0,0,0,0,0,0,0,0,0,0
7266,Yangon,350501,3,350500,Luar Negeri,350000,Myanmar,350500,0,0,...,0,7,7,14,2,5,7,13,12,25
7267,Bandar Seri Begawan,351501,3,351500,Luar Negeri,350000,Brunei Darussalam,351500,0,0,...,0,0,0,0,0,0,0,5,6,11


## Scrape Lvl 4

In [ ]:
# Create folder 'lvl_4' if it doesn't exist
lvl_4_dir = 'lvl_4'
if not os.path.exists(lvl_4_dir):
    os.makedirs(lvl_4_dir)

In [49]:
ls_df_lvl_4 = []
df_to = df_lvl_3.iloc[3075:,:]
for _, row in tqdm(df_to.iterrows(), desc="Processing districs", total=len(df_to)):
    kode_wilayah = row['kode_wilayah']
    api_url_lvl_4 = 'https://dapo.dikdasmen.go.id/rekap/progresSP?id_level_wilayah=3&kode_wilayah=' + kode_wilayah.strip() + '&semester_id=' + semester_id
    r4 = S.get(api_url_lvl_4)
    #print(r2.text)
    try:
        temp_df_lvl_4 = pd.DataFrame(r4.json())
        with open(os.path.join(lvl_4_dir, kode_wilayah + '.json'), 'w') as f:
            json.dump(r4.json(), f, indent=2)

        ls_df_lvl_4.append(temp_df_lvl_4)
    except: pass
    #all_kode_wilayah.append(kode_wilayah)
    time.sleep(0.5)

Processing districs:   0%|          | 0/4194 [00:00<?, ?it/s]

: 

: 

In [46]:
df_lvl_4_part = pd.concat(ls_df_lvl_4, ignore_index=True)

In [48]:
df_lvl_4_part

,nama,sekolah_id,npsn,jumlah_kirim,ptk,pegawai,pd,rombel,jml_rk,jml_lab,...,before_tempat_bermain_olahraga,after_tempat_bermain_olahraga,before_ruang_tu,after_ruang_tu,before_ruang_konseling,after_ruang_konseling,before_ruang_osis,after_ruang_osis,before_bangunan,after_bangunan
0,TK NEGERI PEMBINA,B69535A3-06CC-49DB-B0D8-5B4B1CF7C591,69893404,3.0,8.0,5.0,118.0,8.0,7.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KB FADHILUL HAYAT,5EBAE96A-EAEC-4EDA-B643-21EB67458813,69985297,2.0,2.0,1.0,15.0,2.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TK ABA ARIDWAN,2FB42DA0-29D7-4681-A2D3-F8D8C2F6005B,69908239,6.0,4.0,1.0,31.0,3.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TK AL NUR,437A5859-E89A-46B2-BE3D-033133D7F3D2,20268628,6.0,5.0,1.0,49.0,3.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TK AR ROYYAN,36B26361-6933-48F7-9DFC-1801DDCD20B4,69915300,3.0,1.0,1.0,27.0,2.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254649,SMKN 6 BANDAR LAMPUNG,F94BBA89-A0DE-44B4-BA47-4177F4A99DF8,10809707,36.0,58.0,11.0,688.0,32.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254650,SMKS TRI KARYA UTAMA,59DD2F93-094E-4EA7-9CAF-B6DCEA0F9894,69759221,15.0,12.0,2.0,287.0,10.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254651,PKBM DEWI SARTIKA,1C1F2FEB-A45E-4566-B7EF-56316106E3BD,P9908477,17.0,6.0,1.0,98.0,6.0,7.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254652,PKBM KARANG GEMILANG,73F61D85-1946-4875-9FBD-6AC36F86249F,P9908478,20.0,4.0,2.0,250.0,12.0,9.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_lvl_4 = pd.concat(ls_df_lvl_4, ignore_index=True)

In [ ]:
df_lvl_4

,nama,kode_wilayah,id_level_wilayah,mst_kode_wilayah,induk_provinsi,kode_wilayah_induk_provinsi,induk_kabupaten,kode_wilayah_induk_kabupaten,tk_pd_laki,tk_pd_perempuan,...,slb_pd,sd_pd_laki,sd_pd_perempuan,sd_pd,smp_pd_laki,smp_pd_perempuan,smp_pd,total_pd_laki,total_pd_perempuan,total_pd
0,Kec. Cibinong,020521,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,1453,1395,...,379,19012,17629,36641,8673,8314,16987,44507,41693,86200
1,Kec. Cileungsi,020518,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,1163,1142,...,136,19021,18022,37043,7798,7379,15177,38823,36237,75060
2,Kec. Gunungputri,020519,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,1057,978,...,0,16153,14662,30815,6060,5714,11774,31183,28051,59234
3,Kec. Bojong Gede,020522,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,977,905,...,0,12985,12272,25257,5250,4957,10207,27075,25475,52550
4,Kec. Citeureup,020520,3,020500,Prov. Jawa Barat,020000,Kab. Bogor,020500,421,479,...,0,10293,9698,19991,4558,4146,8704,21680,20046,41726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7264,Hong Kong,351401,3,351400,Luar Negeri,350000,Cina,351400,0,0,...,0,0,0,0,0,0,0,4,49,53
7265,Beijing,351402,3,351400,Luar Negeri,350000,Cina,351400,0,0,...,0,0,0,0,0,0,0,0,0,0
7266,Yangon,350501,3,350500,Luar Negeri,350000,Myanmar,350500,0,0,...,0,7,7,14,2,5,7,13,12,25
7267,Bandar Seri Begawan,351501,3,351500,Luar Negeri,350000,Brunei Darussalam,351500,0,0,...,0,0,0,0,0,0,0,5,6,11
